In [ ]:
# %run convert_ipynb_to_py_files.ipynb

In [1]:
import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Resize, Compose, ToTensor, Normalize

import argparse
import os
import math 
import skimage
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import time
import pickle

from datetime import datetime
from pathlib import Path

# from data_classes.py_files.custom_datasets import *
from data_classes.py_files.data_classes import *
from data_classes.py_files.new_dataset import *
# 
from model_classes.py_files.cnn_model import *
from model_classes.py_files.pigan_model import *

from functions import *

%matplotlib qt

Imported Project and Show_images classes.
Imported CNN model.
Imported PI-Gan model.
----------------------------------
Using device for training: cuda
----------------------------------
Loaded all helper functions.


/home/ptenkaate/.local/lib/python3.6/site-packages/matplotlib/backends/qt_editor/figureoptions.py:11: MatplotlibDeprecationWarning: 
The support for Qt4  was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
  from matplotlib.backends.qt_compat import QtGui


#### Import classes

In [2]:
DEVICE = set_device()

print('----------------------------------')
print('Using device for training:', DEVICE)
print('----------------------------------')

----------------------------------
Using device for training: cuda
----------------------------------


# Set the run you want to evaluate 

In [3]:
run = sorted(os.listdir(path='saved_runs'))[-1]
# run = "pi-gan 21-04-2021 06:39:36 "

ARGS = load_args(run)
# ARGS.dataset = "full"
# ARGS.shuffle = False

print(run)
print(vars(ARGS))

Changed param 	epochs: 100.
Changed param 	acc_steps: 64.
Changed param 	eval_every: 5.
Changed param 	cnn_setup: 17.
pi-gan 23-04-2021 08:16:09 
{'name': '', 'pretrained': None, 'pretrained_best': 'train', 'dataset': 'small', 'norm_min_max': [0, 1], 'seed': 34, 'epochs': 100, 'acc_steps': 64, 'eval_every': 5, 'shuffle': True, 'n_coords_sample': 5000, 'cnn_setup': 17, 'dim_hidden': 256, 'siren_hidden_layers': 3, 'first_omega_0': 30.0, 'hidden_omega_0': 30.0, 'cnn_lr': 0.0001, 'siren_lr': 0.0001, 'mapping_lr': 0.0001, 'cnn_wd': 0, 'siren_wd': 0, 'mapping_wd': 0}


# Evaluation

In [4]:
##### data preparation #####
train_dl, val_dl, test_dl = initialize_dataloaders(ARGS)

Train subjects: 1026
Val subjects: 18
Test subjects: 18


#### Load models

In [5]:
models, optims = load_models_and_optims(ARGS)

CNN(
  (model): Sequential(
    (0): Conv3d(1, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (1): ReLU()
    (2): LayerNorm((24, 64, 64), eps=1e-05, elementwise_affine=True)
    (3): Conv3d(16, 16, kernel_size=(5, 5, 5), stride=(2, 2, 2), padding=(2, 2, 2))
    (4): ReLU()
    (5): LayerNorm((12, 32, 32), eps=1e-05, elementwise_affine=True)
    (6): Conv3d(16, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (7): ReLU()
    (8): LayerNorm((12, 32, 32), eps=1e-05, elementwise_affine=True)
    (9): Conv3d(32, 32, kernel_size=(5, 5, 5), stride=(2, 2, 2), padding=(2, 2, 2))
    (10): ReLU()
    (11): LayerNorm((6, 16, 16), eps=1e-05, elementwise_affine=True)
    (12): Conv3d(32, 64, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(2, 2, 2))
    (13): ReLU()
    (14): LayerNorm((6, 16, 16), eps=1e-05, elementwise_affine=True)
    (15): Conv3d(64, 64, kernel_size=(5, 5, 5), stride=(2, 2, 2), padding=(2, 2, 2))
    (16): ReLU()
    (17): LayerNorm((3,

In [6]:
best_loss = "val"

for model in models.keys():
    models[model].load_state_dict(torch.load(f"saved_runs/{run}/{model}_{best_loss}.pt"))

In [7]:
##### loss function #####
criterion = nn.BCELoss()    

In [8]:
def scroll_through_output(dataloader, shape=(64, 64, 24), first=100):
    pcmras = masks = outs = torch.Tensor([])
    titles = []

    for idx, subj, proj, pcmra, coords, pcmra_array, mask_array in dataloader: 
#         print(pcmra.shape)

        siren_out = get_complete_image(models, pcmra, coords)
        loss = criterion(siren_out, mask_array) 

        pcmras = torch.cat((pcmras, pcmra_array.cpu().view(shape).detach()), 2)
        masks = torch.cat((masks, mask_array.cpu().view(shape).detach()), 2)
        outs = torch.cat((outs, siren_out.cpu().view(shape).detach()), 2)

        titles += [f"{idx.item()} {subj[0]} {proj[0]}, loss:, {round(loss.item(), 4)}" for i in range(shape[2])]
        
        if idx >= first: 
            break

    return Show_images(titles, (pcmras.numpy(), "pcmras"), (masks.numpy(), "masks"), (outs.numpy(), "outs"))

In [9]:
window = scroll_through_output(test_dl)